In [36]:
from const import *

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings


from catboost import CatBoostRegressor
from catboost import Pool

warnings.simplefilter('ignore')

In [ ]:
data_path = '../1.data/'
train_file_name = 'train.csv'
test_file_name  = 'test.csv'

In [ ]:
def creansing_dataset(df):
    # year
    is_error_year = df['year'] > 2023
    df.loc[is_error_year, 'year'] = df[is_error_year]['year'] - 1000
    # size
    df['size'] = df['size'].replace(size_replace)
    # manufacturer
    df['manufacturer'] = df['manufacturer'].replace(manufactuer_replace)

    return df

In [ ]:
train_df = pd.read_csv(f'{data_path}{train_file_name}')
test_df = pd.read_csv(f'{data_path}{test_file_name}')
master_df = pd.concat([train_df, test_df])

In [ ]:
non_cols = ['id', 'price']
num_cols = ['year', 'odometer']
cat_cols = [col for col in train_df.columns if col not in non_cols + num_cols]

In [ ]:
def creansing_dataset(df):
    # year
    is_error_year = df['year'] > 2023
    df.loc[is_error_year, 'year'] = df[is_error_year]['year'] - 1000
    # size
    df['size'] = df['size'].replace(size_replace)
    # manufacturer
    df['manufacturer'] = df['manufacturer'].replace(manufactuer_replace)

    return df


def create_num_features(df):
    df['elapsed_year'] = 2023 - df['year']
    del df['year']

    return df


def create_cat_features(df):
    for col in cat_cols:
        df[col] = df[col].factorize()[0]
        df[col] = df[col].astype('category')
    return df


def create_features_main(df):
    df = creansing_dataset(df)
    df = create_num_features(df)
    df = create_cat_features(df)

    return df

In [ ]:
master_df = create_features_main(master_df)
train_df = master_df[~master_df['price'].isnull()].copy()
test_df  = master_df[master_df['price'].isnull()].copy()

In [44]:
def built_catboost_model(cat_features, Xt, yt, Xe, ye):
    model = \
        CatBoostRegressor(
            learning_rate = 0.1,
            iterations=300,
            loss_function = 'MAE',
            early_stopping_rounds = 30,
            random_seed = 42
        )

    model.fit(
        X = Xt, y = yt,
        eval_set = (Xe, ye),
        cat_features = cat_features, 
        use_best_model = True, 
        verbose = 50
    )

    return model

In [49]:
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

In [43]:
train_index, valid_index = train_test_split(train_df['id'].values, test_size=0.25)

x_train = train_df.iloc[train_index].drop(columns='price')
y_train = train_df.iloc[train_index]['price']
x_valid = train_df.iloc[valid_index].drop(columns='price')
y_valid = train_df.iloc[valid_index]['price']


In [45]:
model = built_catboost_model(
    cat_cols, x_train, y_train, x_valid, y_valid
)

0:	learn: 7589.7845121	test: 7525.4996604	best: 7525.4996604 (0)	total: 193ms	remaining: 57.8s
50:	learn: 5555.3850541	test: 5560.9567771	best: 5560.9567771 (50)	total: 2.45s	remaining: 12s
100:	learn: 5473.7228319	test: 5539.3046172	best: 5539.3046172 (100)	total: 4.55s	remaining: 8.96s
150:	learn: 5404.8384738	test: 5532.7261777	best: 5532.7261777 (150)	total: 6.78s	remaining: 6.69s
200:	learn: 5354.6938847	test: 5527.7875062	best: 5527.7875062 (200)	total: 9.11s	remaining: 4.49s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 5527.616494
bestIteration = 201

Shrink model to first 202 iterations.


In [50]:
mean_absolute_error(
    model.predict(x_valid), y_valid)

5527.6164950750845